In [2]:
#calculating work orders closed on time in facilities maintenance division

 - __Measure Description:__This KPI provides information on  how long it takes to close facility maintenance work orders .  It measures the average work order duration of each  individual work order within in each of the varous work order problem types and captures both the duration of each specific work order as well as the average work order duration categorically for each work order problem type.  Determining which and the total number of individual work orders that exceed the average duration for their respective problem type provides insight on the percentage of work orders closed below and above normal duration.   (This approach allows the department to compare apples to apples with respect to work order workload and duration because of natural variations in the amount of time needed to close a work order that are directly related to the type of work order request.  For example, it will take longer to close a roofing work order than a door knob repair or a custodial work order given the nature of the work and potential for waiting time for parts.)
 - __Data Source:__ Source of this data is the Archibus--the DGS Facility Maintenance work order management software.  Below is an example to illustrate method of calcluation.  <br>Total # of work orders with duration at or below the average duration for its specific work order problem type = TWOOT.  <br>Total # of work orders in fiscal year = TWOFY.   <br>The measure is ((TWOOT/TWOFY)x.01)
 - __Purpose:__ This measure allows us to effectively compare activity on work orders against quarterly, yearly or other periods. This indicator has helped to develop problem type specific and Facility Maintenance shop specific strategies to address strengths, weaknesses and opportunities.  Use of this indicator can support the future development of more granulalar duration standards by category.  It also informs the division on the potential impact that factors such as  parts and equipment delivery may have on customer service perceptions. 

In [3]:
# break up into three sections:
# 1. data analysis to make calculations for work orders closed on time + functions that edit data types/Fiscal Year
# 2. plot files to plot average duration top 10 work orders over a year in bubble chart?
# 3. file that brings all functions together

In [4]:
import pandas as pd
import numpy as np

#packages for updating archibus-connected excel sheet
#import win32com.client
from time import sleep

# import data

#function to automatically update archibus connected excel sheet
#and return a df of updated data
#create function to give an updated archibus dataframe
def ArchibusUpdate():
    # Start an instance of Excel
    xlapp = win32com.client.DispatchEx("Excel.Application")
    
    # Open the workbook in said instance of Excel, workbook will refresh on open
    wb = xlapp.workbooks.open("Z:\\BLima\\Archibus Dump 04-27-17-Database connected.xlsx")

    #sleep to allow time for document to refresh
    sleep(30)

    #save the document
    wb.Save()

    #sleep to allow time for document to save
    sleep(30)

    #quit excel
    xlapp.Quit()
    
    #save updated excel sheet as new dataframe = df
    df=pd.read_excel("Z:\\BLima\\Archibus Dump 04-27-17-Database connected.xlsx")
    
    #print dataframe
    return df

df=ArchibusUpdate()

In [5]:
# df.columns.tolist()

In [6]:
# create dataframe that  includes only relevant columns, adds month/fiscal year columns for duration-related col

def readable_df():
    #df=ArchibusUpdate()
    df = pd.read_csv('./archibus_2017_18.csv', encoding = "ISO-8859-1", low_memory=False)
    #list of column names to be used in updated column
    col = ['bl_id', 
         'site_id',
         'prob_type',
         'completed_by', 
         'cost_total', 
         'date_assigned',
         'date_closed',
         'date_completed',
         'date_requested',
         'manager',
         'time_assigned',
         'time_completed',
         'time_requested',
         'time_stat_chg',
         'po_number',
         'name']
    
    df = df.filter(items=col)
    
    #convert date columns to datetime
    col = df.columns.tolist()
    # create list of columns that only start with date
    dt_col = filter(lambda x: x.startswith(('date', 'time')), col)
   
    #if columns starts with 'date' then convert to datetime object
    for i in dt_col:
        df[i] = pd.to_datetime(df[i])
    
    
    # add in month and fiscal year columns for dates requested-completed
    df['date_requested_month'] = df['date_requested'].dt.month
    df['date_requested_fiscalyear'] = np.where(df['date_requested_month']<7, df['date_requested'].dt.year,df['date_requested'].dt.year+1)
    
    df['date_completed_month'] = df['date_completed'].dt.month
    df['date_completed_fiscalyear'] = np.where(df['date_completed_month']<7, df['date_completed'].dt.year,df['date_completed'].dt.year+1)
    return df
    

In [7]:
df=readable_df()

In [8]:
df.head()

,bl_id,site_id,prob_type,completed_by,cost_total,date_assigned,date_closed,date_completed,date_requested,manager,time_assigned,time_completed,time_requested,time_stat_chg,po_number,name,date_requested_month,date_requested_fiscalyear,date_completed_month,date_completed_fiscalyear
0,B00181,DGS,BUILDING INTERIOR INSPECTION,DELMAR.AUSTIN,55.5,2018-07-24,NaT,2018-07-24,2018-07-24,AFM,NaT,2018-08-09 09:28:46,2018-08-09 09:28:27,NaT,NaN,Northern Community Action Center,7,2019,7,2019
1,B00181,DGS,BATHROOM_FIXT,DELMAR.AUSTIN,2.1,2018-07-24,NaT,2018-07-24,2018-07-24,AFM,NaT,2018-08-09 09:29:28,2018-08-09 09:29:04,NaT,NaN,Northern Community Action Center,7,2019,7,2019
2,B00100,HEALTH,BUILDING INTERIOR INSPECTION,DELMAR.AUSTIN,48.6,2018-07-24,NaT,2018-07-24,2018-07-24,AFM,NaT,2018-08-09 12:13:22,2018-08-09 10:14:26,NaT,NaN,Eastern Health District Building,7,2019,7,2019
3,B00063,DGS,PREVENTIVE MAINT,NaN,0.0,2018-07-23,NaT,2018-07-23,2018-07-23,KAGESTA.CUPID,NaT,2018-08-09 08:59:23,2018-08-09 07:00:07,NaT,NaN,District Court of Maryland People's Court,7,2019,7,2019
4,B00020,BCPD,PLUMB/LEAK,NaN,30.0,2018-07-24,NaT,2018-07-24,2018-07-23,AFM,NaT,2018-08-09 10:09:33,2018-08-09 07:11:45,NaT,NaN,Baltimore City Police Headquarters,7,2019,7,2019


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24474 entries, 0 to 24473
Data columns (total 20 columns):
bl_id                        24474 non-null object
site_id                      24474 non-null object
prob_type                    24474 non-null object
completed_by                 946 non-null object
cost_total                   24474 non-null float64
date_assigned                19829 non-null datetime64[ns]
date_closed                  17567 non-null datetime64[ns]
date_completed               24474 non-null datetime64[ns]
date_requested               24474 non-null datetime64[ns]
manager                      24474 non-null object
time_assigned                0 non-null datetime64[ns]
time_completed               24474 non-null datetime64[ns]
time_requested               24474 non-null datetime64[ns]
time_stat_chg                0 non-null datetime64[ns]
po_number                    3842 non-null object
name                         24474 non-null object
date_requested_month 

In [10]:
#calculate durations of each work order by hours and days

def wo_durations():
    df=readable_df()
    # calculate the duration of the work order in days
    df['duration_days'] = (df['date_completed']-df['date_requested']).dt.days
    # calculate the duration of the work order in hours
    df['duration_hr'] = ((df['time_completed']-df['time_requested']).dt.seconds)/360
    return df

In [13]:
# calculate whether or not the work order was completed on time
#need to do this per fiscal year

def wo_on_time():
    df=wo_durations()
    
    # calculate the average duration (in days and hours) for each problem type
    df['avg_duration_days'] = df.groupby('prob_type')['duration_days'].transform('mean')
    df['avg_duration_hr'] = df.groupby('prob_type')['duration_hr'].transform('mean')
    
    # if the duration <= the mean, then it's on time
    # create column that calculates if the work order was on time (Y=yes, N=no)
    df['wo_on_time_days'] = np.where(df['avg_duration_days']<=df['duration_days'], 'Y', 'N')
    df['wo_on_time_hr'] = np.where(df['avg_duration_hr']<=df['duration_hr'], 'Y', 'N')
    
    return df



In [14]:
# need to know three scenarios of work orders closed on time
# 1. overall number and percentage of work orders closed on time
# 2. num/perc of work orders closed on time in relation to the actual time
# 3. aggregate df of average durations, probtype + other information (dates, building) for big-picture analysis

In [ ]:
#need to get total number of items per fiscal year = overall_on_time
# need to get total number of Ys per fiscal year
# need to get percentage for fiscal year and store as variable of that fiscal year
# create dataframe that is only fiscal year and % wo closed on time

In [69]:
def overall_on_time(on_time_duration):
    
    # on_time_duration will be either in days or hours
    
    df = wo_on_time()
    
    #new df of only Y to get total count of Y on time
    df_Y = df.loc[df[on_time_duration] == 'Y']
    df_Y = df_Y.groupby('date_requested_fiscalyear')[on_time_duration].agg(['count']).reset_index().\
    rename(columns = {'count':'count_on_time', 'date_requested_fiscalyear': 'fiscal_year'})
    
    # new df to get total amount of work orders for each fiscal year
    df_fy = df.groupby('date_requested_fiscalyear')['bl_id'].agg(['count']).reset_index().\
    rename(columns = {'count':'count_total', 'date_requested_fiscalyear': 'fiscal_year'})
    
    #merge dfs on fiscal year column to bring together
    df_total_fy = df_Y.merge(df_fy, on='fiscal_year')
    df_total_fy['percent_on_time'] = round(((df_total_fy['count_on_time']/df_total_fy['count_total'])*100),2)
    
    return df_total_fy        

In [71]:
overall_on_time('wo_on_time_hr')

,fiscal_year,count_on_time,count_total,percent_on_time
0,2015,14,34,41.18
1,2016,55,94,58.51
2,2017,4123,9456,43.60
3,2018,6583,14406,45.70
4,2019,179,484,36.98


In [ ]:
def prob_type_on_time():
    
    df = wo_on_time()
    
    #makes columns for average duration for that work order for that year
    
    #calculate work orders closed on time for each work order
    #give range of percentage of work order problem types closed on time
    #